In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('dataset2.csv')

print(df.head(), df.shape)

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520 (1337, 7)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.model_selection import GridSearchCV
import math

# separation des features et de la variable cible
X = df.drop('charges', axis=1)
y = df[['charges']]
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {y.shape}''')


# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')

def log_transform(x):
    return np.log(x + 1)

log_transformer = FunctionTransformer(log_transform)

preprocessor_num = Pipeline(steps=[
    ('log', FunctionTransformer(log_transform)),
])

preprocessor_cat = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessor_num, ['age', 'bmi', 'children']),
        ('cat', preprocessor_cat, ['region', 'sex', 'smoker'])
    ]
)


verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 6)
      Y (la variable cible) : (1337, 1)
 verif du split 80 20 
80% du dataset : X train -> (1136, 6), Y train -> (1136, 1)
20% du dataset : X test -> (201, 6), Y test -> (201, 1)


In [3]:
from sklearn.dummy import DummyRegressor

# création du pipeline dummy
pipeline_dummy = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', DummyRegressor(strategy='mean'))])


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement puis predire
# sur l'ensemble de test avec le meme pipeline
pipeline_dummy.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = pipeline_dummy.predict(X_test)
#print(y_pred_dummy)

#test avec les differentes métriques pour le modele dummy : 
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle dummy: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle dummy: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle dummy: {rmse}')

Mean Squared Error du modèle dummy: 145320521.96584958
Coefficient de determination R² du modèle dummy: -0.0027928862543942223
Root Mean Squared Error (RMSE) du modèle dummy: 12054.89618229247


In [4]:
# Modele de regression lineaire

# création du pipeline de prétraitement et du modèle linear regression
pipeline_lr = Pipeline(steps=[('preprocessor', preprocessor),
                              ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                            ('scaler', StandardScaler()),
                           ('regression', LinearRegression())])


# entraîner le pipeline sur les données d'entraînement 
pipeline_lr.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = pipeline_lr.predict(X_test)
print(y_pred)

#test des les differents metriques sur modele linear regression
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle linear regression: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle linear regression: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle linear regression: {rmse}')

[[ 3141.13260706]
 [35296.49048949]
 [ 6501.13260706]
 [40210.49048949]
 [29208.49048949]
 [ 7521.13260706]
 [ 3380.13260706]
 [ 4859.13260706]
 [ 2841.13260706]
 [39118.49048949]
 [10049.13260706]
 [12701.13260706]
 [ 3583.13260706]
 [12675.13260706]
 [ 3115.13260706]
 [ 3027.13260706]
 [ 2032.63260706]
 [ 6425.13260706]
 [ 5656.13260706]
 [19352.49048949]
 [14210.13260706]
 [10501.13260706]
 [13602.13260706]
 [13340.13260706]
 [ 6362.38260706]
 [ 3507.13260706]
 [13137.13260706]
 [11945.13260706]
 [ 3710.13260706]
 [ 3160.13260706]
 [ 4599.13260706]
 [12351.13260706]
 [10719.13260706]
 [21168.49048949]
 [30236.49048949]
 [49050.49048949]
 [ 5393.13260706]
 [12206.13260706]
 [17467.13260706]
 [11357.13260706]
 [ 5807.13260706]
 [ 5905.13260706]
 [28440.49048949]
 [13360.49048949]
 [30794.49048949]
 [ 3363.13260706]
 [11245.13260706]
 [12194.13260706]
 [41524.49048949]
 [11755.13260706]
 [ 6825.13260706]
 [41960.49048949]
 [15441.13260706]
 [13853.13260706]
 [ 2533.13260706]
 [15467.13

In [5]:
# Calculez le RMSE (la racine carrée du MSE) pour obtenir une idée de l'erreur en termes des unités d'origine de vos données. 
# Comparez le RMSE à la moyenne des 'charges' (df['charges'].mean()) pour évaluer l'erreur relative.

# RMSE plus élevé que la moyenne : le modèle a du mal à capturer la variation et les tendances dans les données. 
# Il prédit généralement des valeurs qui sont relativement éloignées des valeurs réelles.

# Erreur systématique :  le modèle a tendance à sous-estimer ou surestimer systématiquement les valeurs cibles.

# Modèle insuffisamment performant : Un RMSE élevé peut également indiquer que le modèle que vous utilisez n'est pas suffisamment complexe 
# pour capturer les relations présentes dans les données. Cela peut signifier que le modèle ne tient pas compte de toutes les caractéristiques
# pertinentes ou que la stratégie de prédiction choisie (comme la moyenne) n'est pas adaptée aux données.

# En général, un bon modèle de régression devrait avoir un RMSE inférieur à la moyenne de la variable cible, 
# car cela indiquerait que le modèle est capable de faire des prédictions qui se rapprochent des valeurs réelles en moyenne. 
# L'objectif est de minimiser le RMSE pour obtenir des prédictions précises.

print(df['charges'].mean())
print(rmse)

#donc on en résume que le modele dummy est tres mauvais, et c'est normal car le modele dummy 
#en général, le modèle dummy est un modèle très simple et naïf qui est utilisé comme baseline 
#pour évaluer la performance d'autres modèles plus sophistiqués.

13279.121486655948
3726.597247817469


donc on en résume que le modele dummy est tres mauvais, et c'est normal car le modele dummy <br>
est un modèle très simple et naïf qui est utilisé comme baseline pour évaluer la performance d'autres modèles plus sophistiqués

In [6]:
# AVEC LE Modele de regression LASSO

# création du pipeline de prétraitement et du modèle LASSO
pipeline_lasso = Pipeline(steps=[('preprocessor', preprocessor),
                            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                            ('scaler', StandardScaler()),
                           ('regression', Lasso())])

#test de différents hyperparam d'alpha pour le lasso
param_grid_lasso = {
    'regression__alpha': [0.1, 1, 10, 100]  
}

### grid search
grid_lasso = GridSearchCV(pipeline_lasso, param_grid_lasso, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_lasso.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_lasso.predict(X_test)
# print(y_pred)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle lasso: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle lasso: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle lasso: {rmse} \n rappel moyenne charge : 13279')



/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.035e+10, tolerance: 1.315e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.130e+10, tolerance: 1.298e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

Mean Squared Error du modèle lasso: 13672195.929661542
Coefficient de determination R² du modèle lasso: 0.9056542005749
Root Mean Squared Error (RMSE) du modèle lasso: 3697.5932617936146 
 rappel moyenne charge : 13279


/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.213e+10, tolerance: 1.667e+07
  model = cd_fast.enet_coordinate_descent(


In [7]:
# Modele de regression RIDGE

# création du pipeline de prétraitement et du modèle RIDGE
pipeline_ridge = Pipeline(steps=[('preprocessor', preprocessor),
                            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                            ('scaler', StandardScaler()),
                           ('regression', Ridge())])


#test de différents hyperparam d'alpha pour le lasso
param_grid_ridge = {
    'regression__alpha': [0.01, 0.1, 1, 10, 100]  
}

### grid search
grid_ridge = GridSearchCV(pipeline_ridge, param_grid_ridge, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_ridge.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_ridge.predict(X_test)
# print(y_pred)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle ridge: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle ridge: {rmse} \n rappel moyenne charge : 13279')



Coefficient de determination R² du modèle ridge: 0.9047634612377543
Root Mean Squared Error (RMSE) du modèle ridge: 3715.0071498164157 
 rappel moyenne charge : 13279
